In [22]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from pathlib import Path
import scipy.sparse as sparse
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k

In [56]:
df_sub = pd.read_csv(Path.cwd() / 'data' / 'sample_submission.csv')

In [54]:
df_train = pd.read_csv(Path.cwd() / 'data' / 'train.csv')

/home/alex/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 857895 entries, 0 to 857894
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   overall         857895 non-null  float64
 1   verified        857895 non-null  bool   
 2   reviewTime      857895 non-null  object 
 3   asin            857895 non-null  object 
 4   reviewerName    857719 non-null  object 
 5   reviewText      857616 non-null  object 
 6   summary         857725 non-null  object 
 7   unixReviewTime  857895 non-null  int64  
 8   vote            118740 non-null  object 
 9   style           444406 non-null  object 
 10  image           7142 non-null    object 
 11  userid          857895 non-null  int64  
 12  itemid          857895 non-null  int64  
 13  rating          857895 non-null  float64
dtypes: bool(1), float64(2), int64(3), object(8)
memory usage: 85.9+ MB


In [5]:
df_train.sample()

,overall,verified,reviewTime,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image,userid,itemid,rating
548782,5.0,True,"11 30, 2014",B00MRAQUM8,S.L.K.,Good quality and packaged well.,Five Stars,1417305600,NaN,NaN,NaN,83412,30249,1.0


Проверим на дубликаты

In [9]:
res = df_train.duplicated().value_counts()

In [14]:
res[0]/(res[1]*100)

0.28623446132596686

Почти три процента дубликатов. 

In [6]:
df_test = pd.read_csv(Path.cwd() / 'data' / 'test.csv')

/home/alex/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [7]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285965 entries, 0 to 285964
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   verified        285965 non-null  bool  
 1   reviewTime      285965 non-null  object
 2   asin            285965 non-null  object
 3   reviewerName    285918 non-null  object
 4   unixReviewTime  285965 non-null  int64 
 5   vote            39462 non-null   object
 6   style           147680 non-null  object
 7   image           2368 non-null    object
 8   userid          285965 non-null  int64 
 9   itemid          285965 non-null  int64 
 10  Id              285965 non-null  int64 
dtypes: bool(1), int64(4), object(6)
memory usage: 22.1+ MB


In [8]:
df_test.duplicated().value_counts()

False    285965
dtype: int64

In [9]:
df_test.sample()

,verified,reviewTime,asin,reviewerName,unixReviewTime,vote,style,image,userid,itemid,Id
1540,True,"06 23, 2015",B000E1DSTK,JoAnn,1435017600,NaN,{'Style:': ' Cucumber Ranch'},NaN,59569,1366,1540


In [16]:
df_train.drop_duplicates(inplace=True)

In [17]:
train_data, test_data = train_test_split(df_train, random_state=32, shuffle=True)

In [18]:
ratings_coo = sparse.coo_matrix((train_data['rating'].astype(int),
                                 (train_data['userid'],
                                  train_data['itemid'])))

In [43]:
NUM_THREADS = 8 #число потоков
NUM_COMPONENTS = 60 #число параметров вектора 
NUM_EPOCHS = 25 #число эпох обучения

model = LightFM(learning_rate=0.1, loss='logistic',
                no_components=NUM_COMPONENTS)
model = model.fit(ratings_coo, epochs=NUM_EPOCHS, 
                  num_threads=NUM_THREADS)

In [44]:
preds = model.predict(test_data.userid.values,
                      test_data.itemid.values)

In [45]:
roc_auc_score(test_data.rating,preds)

0.7252626451418033

In [46]:
len(preds)

207234

In [49]:
preds.min(), preds.max()

(-35.288516998291016, 38.3861083984375)

In [50]:
normalized_preds = (preds - preds.min())/(preds - preds.min()).max()

In [52]:
normalized_preds.min(), normalized_preds.max()

(0.0, 1.0)

In [57]:
df_sub['rating']= normalized_preds

In [58]:
df_sub.to_csv('submission_log.csv', index=False)